In [85]:
import shutil
import pandas as pd 
from pathlib import Path
from datetime import datetime

In [86]:
class File():
    """ Stores information about a file"""
    def __init__(self, path): 
        self.path = str(path)
        self.parent = str(path.parent)
        self.name = str(path.stem)
        self.type = str(path.suffix)
        self.size = int(path.stat().st_size)
        self.time = float(path.stat().st_mtime)
        self.date = str(datetime.fromtimestamp(self.time).strftime('%Y-%m-%d %H:%M'))

    def __str__(self):
        return f"Object with Filename: {self.name} and Size: {self.path.stat().st_size} ++"

In [87]:
class FileFilter(File):
    """ TODO """
    def __init__(self, path, min_file_size=0): #MB # TODO 
        
        paths = [f for f in list(Path(path).glob('**/*')) if f.is_dir() == False]
        _dict = []
        for path in paths:
            if path.stat().st_size >= min_file_size*1024*1024:
                _dict.append(File(path).__dict__)

        self.dataframe = pd.DataFrame(_dict)
        self.min_file_size = min_file_size

    def __str__(self):
        return f"FileFilter object with df len of: {len(self.dataframe)} and filesize >= {self.min_file_size}"

    def get_dataframe(self):
        return self.dataframe

    # NOTE: If a files is duplicated now only one ('first'?) value is copied to the sorted folder
    # TODO: Add size option
    # TODO BUG / BAD
    def sort_files(self, subset, type, save=False, target='type_sorted', limit = 5):
        result = self.dataframe[subset].str.contains(type, case=False)
        index = result[result].index
        df = self.dataframe.iloc[index].reset_index(drop=True)
        
        # Create folder if save is true
        if save:
            for n in range(limit):
                if n >= limit:
                    raise ValueError(f"Limit for amount of sorted folders is exceeded, folder limit {limit}")
                
                if not Path(f'{target}_{n}').is_dir():
                    Path(f'{target}_{n}').mkdir(exist_ok = True)
                    break
    
            for path in df['path']:
                shutil.copy(path, Path(f'{target}_{n}') / Path(path).name)
        
        return df

    def dups(self, subset):
        result = self.dataframe.duplicated(subset=subset, keep=False)
        index = result[result].index
        return self.dataframe.iloc[index].reset_index(drop=True)

In [88]:
df = FileFilter('files')
print(df)
df.get_dataframe()

FileFilter object with df len of: 10 and filesize >= 0


,path,parent,name,type,size,time,date
0,files\file_system_1\dup_1.txt,files\file_system_1,dup_1,.txt,4,1.665256e+09,2022-10-08 21:05
1,files\file_system_1\dup_2.txt,files\file_system_1,dup_2,.txt,4,1.665852e+09,2022-10-15 18:34
2,files\file_system_1\dup_3.csv,files\file_system_1,dup_3,.csv,2,1.665852e+09,2022-10-15 18:37
3,files\file_system_1\folder_1\dup_4.txt,files\file_system_1\folder_1,dup_4,.txt,0,1.665852e+09,2022-10-15 18:34
4,files\file_system_1\folder_1\singel_1.csv,files\file_system_1\folder_1,singel_1,.csv,4,1.665852e+09,2022-10-15 18:39
5,files\file_system_2\dup_1.txt,files\file_system_2,dup_1,.txt,4,1.665256e+09,2022-10-08 21:05
6,files\file_system_2\dup_2.txt,files\file_system_2,dup_2,.txt,4,1.665852e+09,2022-10-15 18:34
7,files\file_system_2\dup_3.csv,files\file_system_2,dup_3,.csv,2,1.665852e+09,2022-10-15 18:37
8,files\file_system_2\folder_1\dup_4.txt,files\file_system_2\folder_1,dup_4,.txt,0,1.665410e+09,2022-10-10 15:57
9,files\file_system_2\folder_1\singel_2.txt,files\file_system_2\folder_1,singel_2,.txt,28,1.665853e+09,2022-10-15 18:49


In [89]:
df.dups('name')

,path,parent,name,type,size,time,date
0,files\file_system_1\dup_1.txt,files\file_system_1,dup_1,.txt,4,1.665256e+09,2022-10-08 21:05
1,files\file_system_1\dup_2.txt,files\file_system_1,dup_2,.txt,4,1.665852e+09,2022-10-15 18:34
2,files\file_system_1\dup_3.csv,files\file_system_1,dup_3,.csv,2,1.665852e+09,2022-10-15 18:37
3,files\file_system_1\folder_1\dup_4.txt,files\file_system_1\folder_1,dup_4,.txt,0,1.665852e+09,2022-10-15 18:34
4,files\file_system_2\dup_1.txt,files\file_system_2,dup_1,.txt,4,1.665256e+09,2022-10-08 21:05
5,files\file_system_2\dup_2.txt,files\file_system_2,dup_2,.txt,4,1.665852e+09,2022-10-15 18:34
6,files\file_system_2\dup_3.csv,files\file_system_2,dup_3,.csv,2,1.665852e+09,2022-10-15 18:37
7,files\file_system_2\folder_1\dup_4.txt,files\file_system_2\folder_1,dup_4,.txt,0,1.665410e+09,2022-10-10 15:57


In [90]:
df.sort_files('type','.csv')
#for i in range(6):
#    df.sort_files('type','.csv', save=True)

,path,parent,name,type,size,time,date
0,files\file_system_1\dup_3.csv,files\file_system_1,dup_3,.csv,2,1.665852e+09,2022-10-15 18:37
1,files\file_system_1\folder_1\singel_1.csv,files\file_system_1\folder_1,singel_1,.csv,4,1.665852e+09,2022-10-15 18:39
2,files\file_system_2\dup_3.csv,files\file_system_2,dup_3,.csv,2,1.665852e+09,2022-10-15 18:37


In [91]:
def single(subset, df_reference, df_target):
    df = pd.concat([df_reference,df_target]).reset_index(drop=True)
    index = df[subset].drop_duplicates(keep=False).index
    return df.iloc[index].reset_index(drop=True)

df1 = FileFilter('files/file_system_1/', min_file_size=0).get_dataframe()
df2 = FileFilter('files/file_system_2/', min_file_size=0).get_dataframe()

# Unique filename 
df_comp = single('name', df1, df2)
df_comp

# DELETE
df_comp.to_csv("test.csv")

In [92]:
# Unique size
df_comp = single('size', df1, df2)
df_comp

,path,parent,name,type,size,time,date
0,files\file_system_2\folder_1\singel_2.txt,files\file_system_2\folder_1,singel_2,.txt,28,1.665853e+09,2022-10-15 18:49
